In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import string
from sklearn.feature_extraction import stop_words

In [2]:
data = pd.read_csv('GSAF5.csv', encoding = 'latin')
data.head(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
def supprespace(x):
    if type(x) is str:
        return x.strip()
    return x
data = data.applymap(supprespace)

data = data.drop_duplicates() #(subset = ['colonne(s) sur laquelle baser le drop'])

In [4]:
#évalution taille fichier
print(data['Case Number'].count())

5992


In [5]:
#vérifie qu'il n'y a pas de colonnes vides
null_cols = data.isnull().sum()
pourcent_col = null_cols / data['Case Number'].count()*100
print("Poids des valeurs nulles dans chacun des champs")
print(pourcent_col)

Poids des valeurs nulles dans chacun des champs
Case Number                0.000000
Date                       0.000000
Year                       0.000000
Type                       0.000000
Country                    0.717623
Area                       6.708945
Location                   8.277704
Activity                   8.795060
Name                       3.337784
Sex                        9.462617
Age                       44.742991
Injury                     0.450601
Fatal (Y/N)                0.317089
Time                      53.621495
Species                   48.965287
Investigator or Source     0.250334
pdf                        0.000000
href formula               0.016689
href                       0.050067
Case Number.1              0.000000
Case Number.2              0.000000
original order             0.000000
Unnamed: 22               99.983311
Unnamed: 23               99.966622
dtype: float64


In [6]:
#checke le contenu des colonnes presque vides
data[data['Unnamed: 22'].fillna(0) != 0]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1247,2006.06.18,18-Jun-06,2006,Unprovoked,BRAZIL,Pernambuco,"Punta Del Chifre Beach, Olinda",Body boarding,Humberto Pessoa Batista,M,...,NaN,globalsurfnews.com,2006.06.18-Batista.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.06.18,2006.06.18,4746,stopped here,NaN


In [7]:
data[data['Unnamed: 23'].fillna(0) != 0]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4182,1952.07.13,13-Jul-52,1952,Provoked,USA,California,"San Diego, San Diego County",Fishing,"Gerald Howard, on board sportsfishing boat Ter...",M,...,NaN,"L.A. Times, 7/14/1952",1952.07.13-Howard.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.07.13,1952.07.13,1811,NaN,Teramo
5585,1879.03.10,10-Mar-1879,1879,Invalid,AUSTRALIA,New South Wales,Near Sydney,The steamship Bonnie Dundee lost in collision,Cabin boy of the Bonnie Dundee,M,...,NaN,"Star, 3/22/1879",1879.03.10-Bonnie-Dundee.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1879.03.10,1879.03.10,408,NaN,change filename


In [8]:
#les colonnes href, href formula et pdf contiennent des infos redondantes.
#Comparaison des colonnes pour sélectionner laquelle conserver.

#reconstitue un URL à partir de la colonne pdf
urlreconstitue = 'http://sharkattackfile.net/spreadsheets/pdf_directory/' + data['pdf']
#compare cet URL reconstitué avec les contenus des colonnes href et href formula
filthref = ~data['href'].eq(urlreconstitue)
filtformula = ~data['href formula'].eq(urlreconstitue)

#calcul de quelques indicateurs de propreté des champs href
FauxvsPDFhref = data[filthref].size
OKformathref = data['href'].astype(str).apply(lambda x: not x.endswith('.pdf'))
valueshref = data[OKformathref].size

#calcul de quelques indicateurs de propreté du champ href formula
FauxvsPDFformula = data[filtformula].size
OKformatformula = data['href formula'].astype(str).apply(lambda x: not x.endswith('.pdf'))
valuesformula = data[OKformatformula].size

#comparaison de la propreté des 2 champs
print("nb val incohérentes entre le champ pdf et le champ href : " + str(FauxvsPDFhref) + " vs " + str(FauxvsPDFformula) + " avec le champ href formula")
print("nb val qui terminent correctement :" + str(valueshref) + " dans le champ href, vs " + str(valuesformula) + " dans le champ href formula")

nb val incohérentes entre le champ pdf et le champ href : 1296 vs 120 avec le champ href formula
nb val qui terminent correctement :840 dans le champ href, vs 672 dans le champ href formula


In [9]:
##Compte tenu des résultats précédents : on garde le champ hrefformula. => nettoyage de cette colonne avant suppression des autres.

# compléter les cellules vides de href formula par la valeur de href
data['href'] = data['href formula'].fillna(data['href'])

# remplacer les valeurs de href formula lorsqu'elles ne commencent pas comme il faut par celles de href
dataf = data['href'].apply(lambda x: not x.startswith('http'))
values = data[dataf]
data.loc[values.index, 'href formula'] = data.loc[values.index, 'href']

# remplacer les valeurs de href formula lorsqu'elles ne terminent pas comme il faut par celles de href
dataf = data['href'].apply(lambda x: not x.endswith('.pdf'))
values = data[dataf]
data.loc[values.index, 'href formula'] = data.loc[values.index, 'href']

#estimation du nombre d'améliorations possibles encore présentes
data[filtformula == True].shape[0]
print(data[filtformula == True]['href formula'])

241     http://sharkattackfile.net/spreadsheets/pdf_di...
448                    pdf-directory/2013.05.27.b-Ena.pdf
3019                                                  NaN
5906    Q93http://sharkattackfile.net/spreadsheets/pdf...
5991    http://sharkattackfile.net/spreadsheets/pdf_di...
Name: href formula, dtype: object


In [11]:
#récupère les dates en format 01-Janvier-(20)10
data['datecorr2'] = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{4,}-[0-9]{2,4})')
#récupère les dates en format 01-Jan-(20)10
data['datecorr3'] = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{2,4})')
data['datecorr2'] = data['datecorr2'].fillna(data['datecorr3'])
#récupère les dates en format Jan(vier)-(20)10
data['datecorr3'] = data['Date'].str.extract(r'([A-Za-z]{3,}-[0-9]{2,4})')
data['datecorr2'] = data['datecorr2'].fillna('01-' + data['datecorr3'])

In [12]:
def unifdate(date):
    dateunif = []
    for i in date:
        if len(str(i)) == 9:
            i = i.replace('-0','-200').replace('-1','-201')
        if len(str(i)) == 9:
            i = i.replace('-2','-192').replace('-3','-193').replace('-4','-194').replace('-5','-195').replace('-6','-196').replace('-7','-197').replace('-8','-198').replace('-9','-199')
        dateunif.append (i)
    dateformatee = pd.DataFrame(dateunif)
    return dateformatee
data['datecorr2'] = unifdate(data['datecorr2'])
data['datecorr2'] = pd.to_datetime(data['datecorr2'])

In [13]:
#complète les valeurs manquantes par une date fictive
data['datecorr2'] = data['datecorr2'].fillna("01-Jan-2099")
    
data['datecorr2'] = pd.to_datetime(data['datecorr2']).dt.strftime('%Y.%m.%d')

In [14]:
Counter(data['datecorr2']).most_common()

[('2099.01.01', 523),
 ('1960.10.01', 6),
 ('1956.08.01', 6),
 ('2003.10.05', 5),
 ('2001.04.12', 5),
 ('1995.07.28', 5),
 ('1965.06.01', 5),
 ('2015.09.20', 4),
 ('2014.12.28', 4),
 ('2012.06.14', 4),
 ('2008.06.26', 4),
 ('2010.01.09', 4),
 ('2009.07.24', 4),
 ('2008.12.27', 4),
 ('2003.07.01', 4),
 ('1994.07.09', 4),
 ('1970.01.23', 4),
 ('1960.04.01', 4),
 ('1952.07.27', 4),
 ('1906.10.10', 4),
 ('2016.09.18', 3),
 ('2016.07.08', 3),
 ('2016.07.07', 3),
 ('2015.10.17', 3),
 ('2015.06.26', 3),
 ('2015.06.24', 3),
 ('2014.08.27', 3),
 ('2014.06.09', 3),
 ('2014.06.01', 3),
 ('2014.05.14', 3),
 ('2014.03.18', 3),
 ('2013.09.21', 3),
 ('2013.09.01', 3),
 ('2013.07.28', 3),
 ('2013.06.25', 3),
 ('2012.09.02', 3),
 ('2012.07.07', 3),
 ('2012.06.26', 3),
 ('2011.08.17', 3),
 ('2011.06.06', 3),
 ('2010.05.18', 3),
 ('2010.02.06', 3),
 ('2009.03.06', 3),
 ('2009.01.24', 3),
 ('2008.08.30', 3),
 ('2008.06.28', 3),
 ('2008.03.28', 3),
 ('2007.09.30', 3),
 ('2007.08.19', 3),
 ('2006.08.20', 3)

In [15]:
data['Case Number'] = data['Case Number'].apply(lambda x:x.replace("-",".").replace(",",".").replace("/","."))
data['Case Number.1'] = data['Case Number.1'].apply(lambda x:x.replace("-",".").replace(",",".").replace("/","."))
data['Case Number.2'] = data['Case Number.2'].apply(lambda x:x.replace("-",".").replace(",",".").replace("/","."))
data = data.drop_duplicates()

In [16]:
#dataframe qui ne contient que les différences
corr1 = data[data['Case Number'].eq(data['Case Number.1']) == False]
corr2 = data[data['Case Number'].eq(data['Case Number.2']) == False]
listeerr = [corr1, corr2]
corrCaseNum = pd.concat(listeerr)
corrCaseNum = corrCaseNum.drop_duplicates()
corrCaseNum[['Case Number', 'Case Number.1','Case Number.2', 'datecorr2', 'Year']]

,Case Number,Case Number.1,Case Number.2,datecorr2,Year
4,2016.09.15,2016.09.16,2016.09.15,2016.09.16,2016
33,2016.07.14.4,2016.07.14.R,2016.07.14.4,2016.07.14,2016
97,2016.01.24.b,2015.01.24.b,2016.01.24.b,2016.01.24,2016
116,2015.12.23,2015.11.07,2015.12.23,2015.11.07,2015
121,2015.10.28.a,2015.10.28,2015.10.28.a,2015.10.28,2015
3654,1961.09.02.R,1961.09.06.R,1961.09.02.R,1961.09.06,1961
4177,1952.08.05,1952.08.04,1952.08.05,1952.08.05,1952
4719,1934.01.08.R,1934.02.08.R,1934.02.08.R,1934.02.08,1934
5043,1900.00.00.R,1919.00.00.R,1900.00.00.R,2099.01.01,1919
5150,1911.07.31.R,1911.07.31.T,1911.07.31.R,1911.07.31,1911


In [17]:
#à partir des différences, comparaison des dates dans les différents champs :
#on isole les dates
corrCaseNum['DateCaseNb'] = corrCaseNum['Case Number'].apply(lambda x: x[:10])
corrCaseNum['DateCaseNb1'] = corrCaseNum['Case Number.1'].apply(lambda x: x[:10])
corrCaseNum['DateCaseNb2'] = corrCaseNum['Case Number.2'].apply(lambda x: x[:10])
#on crée un nouveau dataframe
#corrCaseNum = pd.concat([DateCaseNb, DateCaseNb1, DateCaseNb2, corrCaseNum['datecorr2']], axis = 1)

In [22]:
comptdate = corrCaseNum[['DateCaseNb', 'DateCaseNb1', 'DateCaseNb2', 'datecorr2']].apply(lambda x: Counter(x).most_common(1)[0], axis = 1)
datemaj = [x[0] for x in comptdate]
print(datemaj)
comptdate
#data.drop(data.loc[corrCaseNum.index,'Case Number'], inplace = True)
#print(data.head(5))

['2016.09.15', '2016.07.14', '2016.01.24', '2015.12.23', '2015.10.28', '1961.09.02', '1952.08.05', '1934.02.08', '1900.00.00', '1911.07.31']


4       (2016.09.15, 2)
33      (2016.07.14, 4)
97      (2016.01.24, 3)
116     (2015.12.23, 2)
121     (2015.10.28, 4)
3654    (1961.09.02, 2)
4177    (1952.08.05, 3)
4719    (1934.02.08, 3)
5043    (1900.00.00, 2)
5150    (1911.07.31, 4)
dtype: object

In [44]:
occurrences = []
for i in comptdate.index:
    data.drop(data['Case Number'][i])
    for j in data['Case Number']:
        if j[:10] == str(comptdate[i][0][:10]):
            occurrences.append(j)
print(occurrences)

KeyError: "['2016.09.15'] not found in axis"

In [19]:
before = corrCaseNum['Case Number']

for x in comptdate.index:
    if comptdate[x][1] >=3:
        if "Reported" in data.loc[x,'Date']:
            corrCaseNum['Case Number'][x] = str(comptdate[x][0]) + '.R'
        else:
            corrCaseNum['Case Number'][x] = str(comptdate[x][0])
after = corrCaseNum['Case Number']
print(before)
print(after)

C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


4         2016.09.15
33      2016.07.14.R
97        2016.01.24
116       2015.12.23
121       2015.10.28
3654    1961.09.02.R
4177      1952.08.05
4719    1934.02.08.R
5043    1900.00.00.R
5150    1911.07.31.R
Name: Case Number, dtype: object
4         2016.09.15
33      2016.07.14.R
97        2016.01.24
116       2015.12.23
121       2015.10.28
3654    1961.09.02.R
4177      1952.08.05
4719    1934.02.08.R
5043    1900.00.00.R
5150    1911.07.31.R
Name: Case Number, dtype: object


In [ ]:
data = pd.read_csv('GSAF5.csv', encoding = 'latin')

In [47]:
#cpt = 0
#for i in corrCaseNum['Case Number']:
#    corrCaseNum['dateCase Number'] = i[:10]
#    corrCaseNum['dateCase Number.1'] = corrCaseNum.loc[corrCaseNum.index[cpt], 'Case Number.1'][:10]
#    corrCaseNum['dateCase Number.2'] = corrCaseNum.loc[corrCaseNum.index[cpt], 'Case Number.2'][:10]
#    if i[:10] == corrCaseNum.loc[corrCaseNum.index[cpt], 'datecorr2'][:10]:
#        print(i[:10])
#    cpt +=1
DumDateCaseNb = pd.get_dummies(corrCaseNum['Case Number'].apply(lambda x: x[:10]))
DumDateCaseNb1 = pd.get_dummies(corrCaseNum['Case Number.1'].apply(lambda x: x[:10]))
DumDateCaseNb2 = pd.get_dummies(corrCaseNum['Case Number.2'].apply(lambda x: x[:10]))
DumDatecorr = pd.get_dummies(corrCaseNum['datecorr2'])
ErrDate1 = pd.merge(DumDateCaseNb, DumDateCaseNb1, left_index=True, right_index=True)
ErrDate2 = pd.merge(DumDateCaseNb2, DumDatecorr, left_index=True, right_index=True)

#corrCaseNum[['Case Number', 'Case Number.1','Case Number.2', 'datecorr2', 'Year']]
#ErrCaseNb = pd.get_dummies(corrCaseNum['Case Number'])
#ErrCaseNb1 = pd.get_dummies(corrCaseNum['Case Number.1'])
#ErrCaseNb2 = pd.get_dummies(corrCaseNum['Case Number.2'])
#ErrDatecorr = pd.get_dummies(corrCaseNum['datecorr2'])
#ErrData1 = pd.merge(ErrCaseNb, ErrCaseNb1, left_index=True, right_index=True)
#ErrData2 = pd.merge(ErrCaseNb2, ErrDatecorr, left_index=True, right_index=True)
#ErrData = pd.merge(ErrData1, ErrData2, left_index=True, right_index=True)
#ErrData.loc[ErrData.index,'False_x']
print(ErrDate1)        

      1900.00.00  1911.07.31_x  1934.01.08  1952.08.05  1961.09.02  \
4              0             0           0           0           0   
33             0             0           0           0           0   
97             0             0           0           0           0   
116            0             0           0           0           0   
121            0             0           0           0           0   
3654           0             0           0           0           1   
4177           0             0           0           1           0   
4719           0             0           1           0           0   
4719           0             0           1           0           0   
4719           0             0           1           0           0   
4719           0             0           1           0           0   
5043           1             0           0           0           0   
5150           0             1           0           0           0   

      2015.10.28_x 

In [ ]:
str(data['Case Number']) == str(data['datecorr2'])

In [ ]:
#supprime les colonnes inutiles
data.drop(['Unnamed: 22', 'Unnamed: 23', 'href formula'], axis = 1, inplace = True)

In [ ]:
#null_cols = data.isnull().sum()
#null_cols[null_cols > 0]
def num_missing(x):
    return sum(x.isnull())
print("Valeurs manquantes par colonne :")
print(data.apply(num_missing, axis=0))

In [ ]:
data.iloc[57,:]

In [ ]:
data.dtypes

In [ ]:
data['Age'] = data['Age'].fillnull(0).astype('int')

In [ ]:
ErrRestantesvspdf = data['href formula'].eq('http://sharkattackfile.net/spreadsheets/pdf_directory/' + data['pdf']) == False
ErrRestantesvsformula = data['href formula'].eq(data['href']) == False
Errpdf = pd.get_dummies(ErrRestantesvspdf)
Errformula = pd.get_dummies(ErrRestantesvsformula)
ErrData = pd.merge(Errpdf, Errformula, left_index=True, right_index=True)#.columns = ['False_pdf','True_pdf','False_formula','True_pdf']
ErrData.loc[ErrData.index,'False_x']
print(ErrData)

In [ ]:
data.describe(include='all').transpose

In [ ]:
Counter(data['Injury']).most_common()

In [ ]:
## HELP

#liste_traitee = re.compile('[a-z]',data['Injury'])
#print(liste_traitee)

bag_of_words = []
caract_virer = ';.!?\n'
liste = re.compile(caract_virer)
print(liste)
liste_injury = [i for i in str(data['Injury']).replace('\n',' ').replace('.',' ').replace('"',' ').replace('\W',' ').split(' ')]
list_of_words = [i.lower() for i in liste_injury if i.lower() not in stop_words.ENGLISH_STOP_WORDS]
bag_of_words = list(set(list_of_words))
print(bag_of_words)


#term_freq = []
#for word in bag_of_words:
#    term_freq.append(str(data['Injury']).count (word))
#print(term_freq)

In [ ]:
data.loc[(data['Country'] == "REUNION"), ['Country', 'Area', 'Location']]

In [ ]:
data.loc[data['Case Number'] == None]

In [ ]:
#fonction qui uniformise le format de date :
#remplace les années sur 2 chiffres en années sur 4 chiffres
#remplace les mois en lettres par des mois en chiffres

def unifdate(date):
    dateunif = []
    for i in date:
        if len(str(i)) == 9:
            i = i.replace('-0','-200').replace('-1','-201')
        if len(str(i)) == 9:
            i = i.replace('-2','-192').replace('-3','-193').replace('-4','-194').replace('-5','-195').replace('-6','-196').replace('-7','-197').replace('-8','-198').replace('-9','-199')
        #i = str(i).replace('Jan','01').replace('Feb','02').replace('Mar','03').replace('Apr','04').replace('May','05').replace('Jun','06').replace('Jul','07').replace('Aug','08').replace('Sep','09').replace('Oct','10').replace('Nov','11').replace('Dec','12')
        #i = str(i).replace('-','/')

        dateunif.append (i)
    dateformatee = pd.DataFrame(dateunif)
    return dateformatee
re.findall(r'([0-9]{2}/[0-9]{2}/[0-9]{4})',str(data['datecorr2']))

data['datecorr2'] = unifdate(data['datecorr2'])
data['datecorr2'] = pd.to_datetime(data['datecorr2'])